In [1]:
import re
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import cleanlab
from cleanlab.classification import CleanLearning
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('/data/ephemeral/code/level2-nlp-datacentric-nlp-05/PMJ/aug_data합친거_특문제거.csv')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
X['input_ids'][0]

tensor([    2,  5108, 19963,  4326,  2309,    23,  2104,  9806, 21688,  4344,
           18,    18,    18,  5482, 23220,  2021,  2200,  4561,  6910,  2031,
         5153,  6262,     3,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [24]:
from sklearn.ensemble import RandomForestClassifier

model_path = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_path)

def bert_vectorize(texts):
    # 입력 텍스트 토큰화 및 텐서 변환
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=128)['input_ids']

X = bert_vectorize(data["text"].tolist())
y = data["target"]
# 학습/검증 데이터 분할

# 분류 모델 설정 (로지스틱 회귀 사용)
model = RandomForestClassifier(
    n_estimators=100,        # 트리의 개수
    max_depth=10,            # 트리의 최대 깊이
    class_weight="balanced", # 언밸런스한 데이터에 대한 가중치 자동 설정
    random_state=42
)

# Cleanlab 설정
clean_learning = CleanLearning(model)
clean_learning.fit(X, y)

# 라벨 오류가 있는 샘플 탐지
label_issues = clean_learning.get_label_issues()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [22]:
label_issues['given_label'].value_counts()

given_label
3    1548
0    1524
6    1488
1    1487
2    1439
4    1416
5    1300
Name: count, dtype: int64

In [25]:
label_issues['predicted_label'].value_counts()

predicted_label
4    1983
6    1872
2    1517
1    1416
3    1227
5    1104
0    1083
Name: count, dtype: int64

In [ ]:
MODEL_NAME = "/data/ephemeral/home/.cache/huggingface/hub/models--DeepMount00--Llama-3.1-8b-Ita/snapshots/2fb3019939a71ff8986ea8ef4bb1bddd1c39baf3"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16).eval()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
def generate_answer(prompt):
    
    messages = [
        {"role": "user", "content": prompt},
    ]       
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=True,
                                          temperature=1)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return decoded[0]

In [6]:
data2 = data.copy()

In [7]:
data2.head(5)

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5


# 나중에 영어 비율 높은거 없애는 코드

In [52]:
def english_ratio(text):
    # 영어 알파벳(a-z, A-Z)만 포함된 문자열을 찾아서 길이를 계산
    english_chars = len(re.findall(r'[a-zA-Z]', text))
    total_chars = len(text)
    return english_chars / total_chars if total_chars > 0 else 0

# 영어 비율과 길이 조건에 따라 데이터 필터링
data['english_ratio'] = data['text'].apply(english_ratio)  # 영어 비율 계산
data['context_length'] = data['text'].apply(len)           # context 길이 계산

# 조건에 맞는 행 필터링 후 삭제
data = data[~((data['english_ratio'] >= 0.4) | (data['context_length'] <= 10))]

# 필요 없는 열 삭제
#data = data.drop(columns=['english_ratio', 'context_length'])

In [51]:
data[data['ID']=='ynat-v1_train_02767']

,ID,text,target,english_ratio,context_length
1685,ynat-v1_train_02767,( politics ),0,0.666667,12


In [8]:
augmented_data = []
for i, (index, row) in enumerate(data.iterrows()):
    ID = row['ID']
    text = row['text']
    target = row['target']
    prompt = (
        "Original headline: " + text + "Please create three new variations of the headline in Korean, while preserving the main topic and category. Each variation should be a natural-sounding full newspaper headline in Korean. Ensure that each sentence is informative, concise, and conveys a similar meaning to the original headline without changing the topic or category. Avoid simple rephrasing or word replacements; instead, aim to capture the original intent with alternative expressions. Finally, provide only the generated variations in the following format:\n generated sentence1: \"[Your first sentence]\"\n generated sentence2: \"[Your second sentence]\"\n generated sentence3: \"[Your third sentence]\""
    )

    answer = generate_answer(prompt)
    new_sentences = answer.split('\n')[-3:]  # 마지막 3개의 문장을 가져옴
    
    for sentence in new_sentences:
        augmented_data.append({
            "ID": ID,                  # 원본 ID 유지
            "text": sentence.strip('"'),  # 증강된 텍스트 추가
            "target": target           # 원본 타겟 유지
        })
augmented_df = pd.DataFrame(augmented_data)
data = pd.concat([data, augmented_df], ignore_index=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask an

In [13]:
data

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5
...,...,...,...
4375,ynat-v1_train_02798,,1
4376,ynat-v1_train_02798,"generated sentence3: ""류현진, 안타타파에 분위기를 높였다 하나 자...",1
4377,ynat-v1_train_02799,"generated sentence1: ""인공지능 시대의 급기항 대책 제시를 위한 국...",3
4378,ynat-v1_train_02799,"generated sentence2: ""지능정보사회 대비 국가 종합대책 마련에 따른...",3


In [14]:
data = data[data['text'].str.len() > 0].reset_index(drop=True)

In [15]:
data

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5
...,...,...,...
4076,ynat-v1_train_02798,"generated sentence2: ""류현진, 구호가 되는 안타는 좋아하나 친구가...",1
4077,ynat-v1_train_02798,"generated sentence3: ""류현진, 안타타파에 분위기를 높였다 하나 자...",1
4078,ynat-v1_train_02799,"generated sentence1: ""인공지능 시대의 급기항 대책 제시를 위한 국...",3
4079,ynat-v1_train_02799,"generated sentence2: ""지능정보사회 대비 국가 종합대책 마련에 따른...",3


In [16]:
def remove_generated_prefix(text):
    # "generated sentence1:", "generated sentence2:", "generated sentence3:" 패턴을 제거
    text = text.strip()
    return re.sub(r'^(generated\s*sentence\s*[1-3]:\s*)', '', text, flags=re.IGNORECASE)

In [17]:
data['text'] = data['text'].apply(remove_generated_prefix)

In [31]:
data2 = pd.read_csv('/data/ephemeral/data/aug_data_특수문자제거.csv')

In [32]:
data2

,ID,text,target,english_ratio,context_length
0,ynat-v1_train_00000,KT SKT LGU 3대 통신사 이용료 인상... 정기요금제로 전환 이용자들 불편 예고,4,0.150943,53.0
1,ynat-v1_train_00001,대한민국 최초의 여성 국회의장으로 선출된 이명박 전 대통령의 딸 이희수 국회의원이 ...,3,0.000000,64.0
2,ynat-v1_train_00002,북한 남북통일을 위한 자주통일의 길을 열어야 한다,2,0.000000,28.0
3,ynat-v1_train_00004,미국 대선 앞두고 선거법 위,6,0.000000,15.0
4,ynat-v1_train_00006,롯데 자이언츠 vs 광주 KIA 타이거즈 프로야구 경기 취소 천재지변으로 인한 경기 중단,1,0.100000,50.0
...,...,...,...,...,...
6162,ynat-v1_train_02792,스타급 선수 몸가치 대폭 오른 이유는,1,NaN,NaN
6163,ynat-v1_train_02792,세계 1 사람들이 장악한 세계스포츠 시장... 선수 등급 상승 효과 공개,1,NaN,NaN
6164,ynat-v1_train_02797,텔레그램 카카오T 등 메신저 네넝 20분3시간간 서비스 중단,4,NaN,NaN
6165,ynat-v1_train_02797,구글 메시자 포함 아시아서 대표 메신저 중 서비스 중단이 속출,4,NaN,NaN


In [33]:
combined_data = pd.concat([data, data2], ignore_index=True)

In [34]:
combined_data.to_csv('aug_data합친거.csv')

In [19]:
def english_ratio(text):
    english_chars = len(re.findall(r'[a-zA-Z]', text))
    total_chars = len(text)
    return english_chars / total_chars if total_chars > 0 else 0

# 영어 비율이 70% 이상인 행 삭제
data = data[data['text'].apply(english_ratio) < 0.7].reset_index(drop=True)

In [23]:
data5 = data.copy()

In [25]:
data5

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5
...,...,...,...
4030,ynat-v1_train_02798,"""류현진, 구호가 되는 안타는 좋아하나 친구가 안타 맞는 것은 싫어한다",1
4031,ynat-v1_train_02798,"""류현진, 안타타파에 분위기를 높였다 하나 자신의 친구가 안타를 맞는 것을 원치 않는다",1
4032,ynat-v1_train_02799,"""인공지능 시대의 급기항 대책 제시를 위한 국가 의제 성의 높여야",3
4033,ynat-v1_train_02799,"""지능정보사회 대비 국가 종합대책 마련에 따른 지속적인 추진 강구 요구",3


In [29]:
data['text'] = data['text'].apply(lambda x: x.split(':', 1)[1].strip() if 'generate' in x else x)

# 결과 확인
data.reset_index(drop=True, inplace=True)

In [27]:
data = data[~data['text'].str.strip().isin(['', ' '])].reset_index(drop=True)

In [30]:
data.to_csv('aug_data_version2.csv', index=False)

In [22]:
data

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5
...,...,...,...
4045,ynat-v1_train_02798,"""류현진, 구호가 되는 안타는 좋아하나 친구가 안타 맞는 것은 싫어한다",1
4046,ynat-v1_train_02798,"""류현진, 안타타파에 분위기를 높였다 하나 자신의 친구가 안타를 맞는 것을 원치 않는다",1
4047,ynat-v1_train_02799,"""인공지능 시대의 급기항 대책 제시를 위한 국가 의제 성의 높여야",3
4048,ynat-v1_train_02799,"""지능정보사회 대비 국가 종합대책 마련에 따른 지속적인 추진 강구 요구",3


In [83]:
data3 = pd.read_csv('/data/ephemeral/code/aug_data.csv')

In [87]:
data3['target'].value_counts()

target
1    922
4    890
5    888
3    878
2    877
6    862
0    850
Name: count, dtype: int64

In [1]:
import pandas as pd
data3 = pd.read_csv('/data/ephemeral/code/level2-nlp-datacentric-nlp-05/PMJ/aug_relabeldata_.csv')

In [5]:
data3

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,2
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5
...,...,...,...
4030,ynat-v1_train_02798,류현진 구호가 되는 안타는 좋아하나 친구가 안타 맞는 것은 싫어한다,1
4031,ynat-v1_train_02798,류현진 안타타파에 분위기를 높였다 하나 자신의 친구가 안타를 맞는 것을 원치 않는다,1
4032,ynat-v1_train_02799,인공지능 시대의 급기항 대책 제시를 위한 국가 의제 성의 높여야,3
4033,ynat-v1_train_02799,지능정보사회 대비 국가 종합대책 마련에 따른 지속적인 추진 강구 요구,3


In [4]:
import re

def remove_unwanted_characters(text):
    # 한글, 영어, 숫자, 한자, 공백, 특정 특수문자 (. ·)만 남기고 제거
    return re.sub(r'[^가-힣a-zA-Z0-9\s\u4e00-\u9fff\.\·]', '', text)

# 데이터프레임의 text 열에 함수 적용
data3['text'] = data3['text'].apply(remove_unwanted_characters)

In [6]:
data3.to_csv('aug_relabeldata_특수제거.csv', index=False)

In [7]:
data4 = pd.read_csv('/data/ephemeral/data/aug_data_특수문자제거.csv')

In [8]:
combined_data = pd.concat([data4, data3], ignore_index=True)

In [9]:
combined_data.to_csv('aug_data합친거_특문제거.csv')

In [91]:
data_fil= pd.read_csv('/data/ephemeral/code/filtered_train_df3.csv')

In [94]:
filtered_data4 = data4[data4['ID'].isin(data_fil['ID'])].copy()

# 'aug_text' 내용을 'text'로 옮기고, 기존 'text'는 무시
filtered_data4['text'] = filtered_data4['aug_text']

# 필요한 열(ID, target, text)만 선택
filtered_data4 = filtered_data4[['ID', 'target', 'text']]

# data3 뒤에 filtered_data4를 concat하여 새로운 DataFrame 생성
result = pd.concat([data3, filtered_data4], ignore_index=True)

In [96]:
result.to_csv('aug_data_특수문자제거+용준이형.csv', index=False)

In [90]:
data4

,ID,text,target,aug_text
0,ynat-v1_train_00000,"KT, 2년 만에 Q2 보급 확대 계획 발표 - 파이낸스타임스",4,"KT, 2년 만에 Q2 보급 확대 계획 발표로 5G 네트워크 도입 가속화할 것 - ..."
1,ynat-v1_train_00001,K찰국 DLwo 로L3한N% 회장 2 T0&}송=,3,K찰국 vs DLwo 로L3한N% 회장 2 T0&}송=
2,ynat-v1_train_00002,"김정연의 북한 방문, 남북 정상회담 가능성에 주목",2,"1. 김정연 북한 지도자, 남한 방문 시 주목할 점들2. 북한 김정연의 남한 방문:..."
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,"갤노트8 주말 27만대 판매로 스마트폰 시장에 활력, 하지만 불법 보조금 문제로 소..."
4,ynat-v1_train_00004,"미대선 앞두고 R2프랑스 단일 정당, 비해 감시 강화",6,R2프랑스가 미대선에 앞서 단일 정당을 구성함에 따라 정치적 경쟁 환경의 변화가 예...
...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,미국 방송사 폭스뉴스의 앵커들에 대한 충성도 평가가 화제가 되고 있다. 여러 전문가...
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,"삼성 갤럭시S9의 정식 출시가 첫 주말에 이루어졌음에도 불구하고, 이통시장의 반응은..."
2797,ynat-v1_train_02797,"텔레그램, 한글 입력 시 2시간 이상 다운... 사용자 분노의 목소리 높아져",4,최근 텔레그램 앱에서 한글 입력 시 2시간 이상 다운되는 문제가 지속되면서 사용자들...
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,"고양이와 거북이 맞대결, 누가 이길까?인터뷰에 나온 한 스포츠 전문가의 말에 따르면..."
